In [1]:
from osrsbox import items_api
import re
import json
import requests
import pandas as pd
import numpy as np

# Proving that there is a problem #

In [2]:
#the three kinds of tradeable chinchompas
chinchompas = ["Chinchompa", "Red chinchompa", "Black chinchompa"] 

In [3]:
#AllItems object
all_items = items_api.load()

In [4]:
def get_select_properties(item_name: str, select_properties: list =["id", "name", "buy_limit", "wiki_url"])->dict:
    """
    Query osrsbox-db for specific keys in the ItemProperties of an item
    """
    properties = all_items.lookup_by_item_name(item_name).construct_json()
    return {p:properties[p] for p in select_properties}

for c in chinchompas:
    print(f"{get_select_properties(c)}\n")

{'id': 9976, 'name': 'Chinchompa', 'buy_limit': None, 'wiki_url': 'https://oldschool.runescape.wiki/w/Chinchompa'}

{'id': 9977, 'name': 'Red chinchompa', 'buy_limit': None, 'wiki_url': 'https://oldschool.runescape.wiki/w/Red_chinchompa'}

{'id': 11959, 'name': 'Black chinchompa', 'buy_limit': 11000, 'wiki_url': 'https://oldschool.runescape.wiki/w/Black_chinchompa'}



So grey chinchompas and red chinchompas have `buy_limit==None`, but black chinchompas have a `buy_limit==11000`

If I log into OSRS, I can check these values at the Grand Exchange or using a RuneLite plugin. Currently (Jan 06, 2021) grey and red chinchompas have a buy limit of 7,000 and black chinchompas have a buy limit of 11,000. We can also verify these same values on the Old School Runescape Wiki.

So clearly, there is a problem with the buy limit on querying from `osrsbox-db` for these 2 items.

# Finding the Cause of the Problem #

Calling `items_api.load()` in the above code creates an instance of `AllItems()`. If we search through the source code for the `AllItems()` class, we see that a dictionary with item properties is created as an attribute of the class (line 42 of `all_items.py`) using a particular JSON file called `items-complete.json`.


Let's take a look at the entry for Chinchompas and Red chinchompas inside `items-complete.json`.

*Note:  The unique Item ID for (grey) Chinchompas is 9976*

In [5]:
#github link to the raw file of `items-complete.json`
url = "https://raw.githubusercontent.com/osrsbox/osrsbox-db/master/osrsbox/docs/items-complete.json"
#this is `items-complete.json` as a Python dictionary
items_complete_dict = requests.get(url).json()

In [6]:
#get the value for chinchompas (id 9976)
#a glance at items-complete.json shows that keys are the item ids as strings
chinchompas_items_complete = items_complete_dict["9976"]
#select a few properties to look at
select = ["id", "name", "buy_limit", "wiki_url"]
{p:chinchompas_items_complete[p] for p in select}

{'id': 9976,
 'name': 'Chinchompa',
 'buy_limit': None,
 'wiki_url': 'https://oldschool.runescape.wiki/w/Chinchompa'}

So the buy limits are incorrect because of the actual data inside `items-complete.json`.

Looking further, we can see `item-buylimits.json` has a JSON object pairing "item name":buy_limit correctly! This file was created (correctly) by the `fetch()` method at `osrsbox-db/scripts/items/item_buylimits.py`. 

So let's see what other tradeables might have been affected, write a method to update `items-complete.json`, then test to see if the changes took place.

# Finding Other Affected Items #

In [7]:
item_buylimits_json = requests.get("https://raw.githubusercontent.com/osrsbox/osrsbox-db/master/data/items/items-buylimits.json").json()

affected_items = {}

for item in items_api.load().__iter__():
    if item.tradeable_on_ge:
        #use a the item ID to search the item in `items-complete.json`
        item_ID = str(item.id)
        #use the item name to search `items-buylimits.json`
        item_name = item.name
        try:
            #buy limit according to `items-complete.json`
            candidate_limit = items_complete_dict[item_ID]["buy_limit"]
            #buy limit according to `item_buylimits.json`
            truth_limit = item_buylimits_json[item_name]
        except KeyError:
            pass
        if candidate_limit != truth_limit:
            affected_items[item_name] = f"Candidate:{candidate_limit}  Truth:{truth_limit}"
affected_items

{'Priest gown': 'Candidate:150  Truth:11000',
 'Shantay pass': 'Candidate:18000  Truth:150',
 'Swamp toad': 'Candidate:13000  Truth:6000',
 'Myre snelm': 'Candidate:150  Truth:11000',
 "Blood'n'tar snelm": 'Candidate:150  Truth:11000',
 'Ochre snelm': 'Candidate:150  Truth:11000',
 'Bruise blue snelm': 'Candidate:150  Truth:11000',
 'Blamish myre shell': 'Candidate:13000  Truth:150',
 'Blamish red shell': 'Candidate:13000  Truth:150',
 'Blamish ochre shell': 'Candidate:13000  Truth:150',
 'Blamish blue shell': 'Candidate:13000  Truth:150',
 'Candle lantern': 'Candidate:None  Truth:10000',
 'Woven top': 'Candidate:150  Truth:4',
 'Shirt': 'Candidate:150  Truth:4',
 'Trousers': 'Candidate:150  Truth:4',
 'Shorts': 'Candidate:150  Truth:4',
 'Skirt': 'Candidate:150  Truth:4',
 'Broodoo shield (10)': 'Candidate:None  Truth:11000',
 'Broodoo shield': 'Candidate:None  Truth:11000',
 'Tribal mask': 'Candidate:150  Truth:13000',
 'Tribal top': 'Candidate:150  Truth:13000',
 'Villager robe': 'C

 # Our Investigation Shows a New Problem! #
 
The above shows that some of our assumptions were incorrect. Take a look at "Lumbridge Teleport", for example...

In [8]:
affected_items["Lumbridge teleport"]

'Candidate:10000  Truth:13000'

This line is saying that the Lumbridge teleport buy limit *should* be set to 13,000, but we currently have it set to 10,000.

However, if we visit [the OSRS Wiki page](https://oldschool.runescape.wiki/w/Lumbridge_teleport_(tablet)) for this item, we clearly see that **the actual buy limit is 10,000**. Logging into the game confirms that 10,000 is the true buy limit, too.

So, our initial thoughts that it was a mismatch in the JSON files was somewhat correct - there *is* a mismatch - however, where a single file is not responsible for the mismatch.

# Our New Solution #

The Old School Wiki has proven throughout this investigation to be reliable, matching with in-game limits at every check.

So, my proposed solution is to build a web scraper to get buy limits from the OSRS Wiki. Then, we can update the corresponding JSON files in the project repository with our scraper values. Finally, we can test our solution using the items we looked at above.

# OSRS Wiki Scraper #

In [9]:
#tabular contents of the page
osrswiki = requests.get("https://oldschool.runescape.wiki/w/Grand_Exchange/Buying_limits").text
#pandas DataFrame of the buy limits table
contents = pd.DataFrame(pd.read_html(osrswiki)[0])
contents

,Item,Buy limit
0,3rd age amulet,8
1,3rd age axe,40
2,3rd age bow,8
3,3rd age cloak,8
4,3rd age druidic cloak,8
...,...,...
3579,Zogre bones,3000
3580,Zombie head,4
3581,Zul-andra teleport,10000
3582,Zulrah's scales,30000


In [10]:
#items we looked at and noticed had problems
known_wrong_items = ["Chinchompa", "Red chinchompa", "Black chinchompa", "Lumbridge teleport"]
#quick visual check that with our scraper the buy limits are correct
[(r, contents[contents["Item"]==r]["Buy limit"].to_numpy()[0]) for r in known_wrong_items]

[('Chinchompa', 7000),
 ('Red chinchompa', 7000),
 ('Black chinchompa', 11000),
 ('Lumbridge teleport', 10000)]

In [11]:
contents["Item"]

0              3rd age amulet
1                 3rd age axe
2                 3rd age bow
3               3rd age cloak
4       3rd age druidic cloak
                ...          
3579              Zogre bones
3580              Zombie head
3581       Zul-andra teleport
3582          Zulrah's scales
3583           Zuriel's staff
Name: Item, Length: 3584, dtype: object

In [12]:
foo = contents.to_dict(orient="dict")
foo

{'Item': {0: '3rd age amulet',
  1: '3rd age axe',
  2: '3rd age bow',
  3: '3rd age cloak',
  4: '3rd age druidic cloak',
  5: '3rd age druidic robe bottoms',
  6: '3rd age druidic robe top',
  7: '3rd age druidic staff',
  8: '3rd age full helmet',
  9: '3rd age kiteshield',
  10: '3rd age longsword',
  11: '3rd age mage hat',
  12: '3rd age pickaxe',
  13: '3rd age platebody',
  14: '3rd age platelegs',
  15: '3rd age plateskirt',
  16: '3rd age range coif',
  17: '3rd age range legs',
  18: '3rd age range top',
  19: '3rd age robe top',
  20: '3rd age robe',
  21: '3rd age vambraces',
  22: '3rd age wand',
  23: 'A powdered wig',
  24: 'Abyssal bludgeon',
  25: 'Abyssal bracelet(5)',
  26: 'Abyssal dagger (p)',
  27: 'Abyssal dagger (p+)',
  28: 'Abyssal dagger (p++)',
  29: 'Abyssal dagger',
  30: 'Abyssal whip',
  31: 'Achey tree logs',
  32: 'Acorn',
  33: 'Adamant 2h sword',
  34: 'Adamant arrow(p)',
  35: 'Adamant arrow(p+)',
  36: 'Adamant arrow(p++)',
  37: 'Adamant arrow',


In [13]:
#write the corrected file

corrected_dict = {}
for i in range(len(foo["Item"])):
    #name = f'"{foo["Item"][i]}"'
    name = r'\"{}\"'.format(foo["Item"][i])
    #limit = f'"{foo["Buy limit"][i]}"'
    limit = r'\"{}\"'.format(foo["Buy limit"][i])
    corrected_dict[name] = int(limit)

corrected_dict

ValueError: invalid literal for int() with base 10: '\\"8\\"'

In [ ]:
f = open("corrected-item-buylimits.json", 'w')
f.write(str(corrected_dict))
f.close()

In [ ]:
#upload the corrected file to the branch

In [ ]:
#write a simple test for the values you know to be broken